# Lecture 1 — Variance

D. Jason Koskinen
Dec. 22, 2015

*(Modernized by J-L Tastet 2019-01-30)*

The exercise herein is to take a .txt file
and read in multiple data sets of X and Y
input and calculate basic statitics quantities.

Do everything in normal python arrays and then
use the numpy converter to put the data into
numpy arrays. Why? Because numpy arrays are hard.

The data set is somehwat of a classic in statistics
and is known as Anscombe's quartet.

Relevant links:
* [Lecture notes](http://www.nbi.dk/~koskinen/Teaching/AdvancedMethodsInAppliedStatistics2019/Lecture1_Basics_ChiSquare.pdf)
* [Dataset](http://www.nbi.dk/%7Ekoskinen/Teaching/AdvancedMethodsInAppliedStatistics2018/data/FranksNumbers.txt)
* NumPy user guide: https://docs.scipy.org/doc/numpy/user/
* NumPy documentation: https://docs.scipy.org/doc/numpy/reference/
* SciPy library documentation: https://docs.scipy.org/doc/scipy/reference/
* String formatting in Python 3:
  * [Good StackOverflow question](https://stackoverflow.com/questions/13945749/string-formatting-in-python-3)
  * https://pyformat.info/
  * ['f-strings' in Python 3.6+](https://realpython.com/python-f-strings/#python-f-strings-the-pesky-details)
* Python 2 countdown: https://pythonclock.org/
* Moving to Python 3: https://python3statement.org/

In [4]:
import io
import numpy as np
import scipy as sp
from scipy import stats as stats

In [15]:
infile = io.open("ex1.dat") # You may need to modify this path

Making an empty array to fill with arrays (of arrays!). Normally arrays of arrays is a bad sign, but it will work out fine this time.

In [16]:
metaArray = []

for line in infile.readlines():
    splLine = line.split()
    if len(splLine) == 3:# This is when the data sets change
        metaArray.append([])
    # end if len()
    if len(splLine) == 0 or (not splLine[0].isdigit()):
        continue
    # end not if
    
    # read in from text is generally a string so make sure
    # to explicitly cast the variable as a float
    x = float(splLine[0])
    y = float(splLine[1])
    metaArray[-1].append([x,y])
# end for line
    
print(metaArray)

[[[10.0, 8.04], [8.0, 6.95], [13.0, 7.58], [9.0, 8.81], [11.0, 8.33], [14.0, 9.96], [6.0, 7.24], [4.0, 4.26], [12.0, 10.84], [7.0, 4.82], [5.0, 5.68]], [[10.0, 9.14], [8.0, 8.14], [13.0, 8.74], [9.0, 8.77], [11.0, 9.26], [14.0, 8.1], [6.0, 6.13], [4.0, 3.1], [12.0, 9.13], [7.0, 7.26], [5.0, 4.74]], [[10.0, 7.46], [8.0, 6.77], [13.0, 12.74], [9.0, 7.11], [11.0, 7.81], [14.0, 8.84], [6.0, 6.08], [4.0, 5.39], [12.0, 8.15], [7.0, 6.42], [5.0, 5.73]], [[8.0, 6.58], [8.0, 5.76], [8.0, 7.71], [8.0, 8.84], [8.0, 8.47], [8.0, 7.04], [8.0, 5.25], [19.0, 12.5], [8.0, 5.56], [8.0, 7.91], [8.0, 6.89]], [[8.0, 6.58], [8.0, 5.76], [8.0, 7.71], [8.0, 8.84], [8.0, 8.47], [8.0, 7.04], [8.0, 5.25], [19.0, 12.5], [8.0, 5.56], [8.0, 7.91], [8.0, 6.89], [10.0, 7.46], [8.0, 6.77], [13.0, 12.74], [9.0, 7.11], [11.0, 7.81], [14.0, 8.84], [6.0, 6.08], [4.0, 5.39], [12.0, 8.15], [7.0, 6.42], [5.0, 5.73], [10.0, 9.14], [8.0, 8.14], [13.0, 8.74], [9.0, 8.77], [11.0, 9.26], [14.0, 8.1], [6.0, 6.13], [4.0, 3.1], [12

Convert the array of arrays of arrays into a list of 2D numpy arrays, so that nice calculations can be made with ease.

We have to use a list instead of a 3D array because the 2D arrays it contains have different dimensions.

To build the list, we use a list comprehension.

In [7]:
a = [np.asarray(arr2d) for arr2d in metaArray]

The analysis is the same for each dataset, so let's write a function.

In [8]:
def compute_chi_squared(a, i):
    """
    a: List of arrays containing all datasets
    i: Index of the dataset for which we want to compute the χ²
    """
    slope = 0.48
    intercept = 3.02
    # The following code 'flattens' the tuple, which then includes
    # the x-values (1st column in the file) as part of the set over
    # which to compute the variance.
    # but actually we just want to compute the variance of the y-values.
    wrong_var = np.var(a[i])
    print(f"Variance for dataset {i}: {wrong_var:.3f} (WRONG VALUE)")

    # The following code tells numpy (via the axis=0) to calculate
    # the variance over the
    # separate data columns (x and y), where we're mostly interested in the
    # variance in y. Also, there are two ways to think of the
    # exercise as written in the lecture notes:
    # A) you are given the line and therefore the degrees of freedom
    # are equal to the number of data points, or
    # B) the variance should be calculated using the 'unbiased'
    # estimator (shown on slide 3) which corrects the
    # degrees of freedom to be N-1. By default numpy uses
    # that the change to the degrees of freedom (ddof) is zero.
    # Ergo, for an unbiased estimator we maybe, possibly, kinda, sort of,
    # should use N-1 stead of N. Also, Troels said that he stressed this
    # in his class, so all of the students from his course should
    # know this.
    biased_var = np.var(a[i], axis=0, ddof=0)[1]
    print(f"Variance for dataset {i}: {biased_var:.3f} (CORRECT VALUE FOR BIASED VARIANCE)")
    unbiased_var = np.var(a[i], axis=0, ddof=1)[1]
    print(f"Variance for dataset {i}: {unbiased_var:.3f} (CORRECT VALUE FOR UNBIASED VARIANCE)")
    linreg = stats.linregress(a[i])
    print(f"linear regression:  y={linreg[0]:0.2f}x + {linreg[1]:0.2f}")
    
    # just get the y-values, i.e. the observed data.
    # Note that this is more easily done if the data sets
    # have the exact numbers of entries, unlike here. The
    # difference is where you put the [:,1] and whether it
    # is necessary to 'recreate' a new numpy array.
    
    observed = sp.array(a[i])[:,1]
    expected = []
    chisq_value     = 0
    chisq_valuewith = 0

    # loop over all the data points in the data set
    # to calculate the expected values of y at each
    # value of x.
    for j in range(0, len(a[i])):
        x = a[i][j][0]
        y = x*slope + intercept
        expected.append(y)
        chisq_value += (y - observed[j])**2/y
        chisq_valuewith += (y - observed[j])**2/(1.22**2)
    # end for x,y
    
    print("chi-squared By hand:    {:.3f}".format(chisq_value))
    print("chi-squared From SciPy: {:.3f}".format(stats.chisquare(observed,expected)[0]))
    print("chi-squared (w/ ±1.22 uncertainty):    {:.3f}".format(chisq_valuewith))
    print("Reduced chi-squared:    {:.3f}".format((chisq_value)/(len(a[i]))))
    print("Reduced chi-squared (w/ ±1.22 uncertainty):    {:.3f}".format((chisq_valuewith)/(len(a[i]))))
    print("\n")

Compute $\chi^2$ for each dataset.

In [9]:
for i in range(0,len(a)):
    compute_chi_squared(a, i)

Variance for dataset 0: 7.438 (WRONG VALUE)
Variance for dataset 0: 3.752 (CORRECT VALUE FOR BIASED VARIANCE)
Variance for dataset 0: 4.127 (CORRECT VALUE FOR UNBIASED VARIANCE)
linear regression:  y=0.50x + 3.00
chi-squared By hand:    1.887


/var/folders/5g/9cr0t2sj5mj8w4wcjj4by3300000gq/T/ipykernel_5622/1295949701.py:43: DeprecationWarning: scipy.array is deprecated and will be removed in SciPy 2.0.0, use numpy.array instead
  observed = sp.array(a[i])[:,1]


ValueError: For each axis slice, the sum of the observed frequencies must agree with the sum of the expected frequencies to a relative tolerance of 1e-08, but the percent differences are:
0.0219222194699037

There is a larger questions here related to calculation of the chi-squared value; we can do it, but if we do not know actually what the data is (money, number of cows, speed of a toddler, etc.) can the chi-squared or the reduced chi-squared tell use anything meaningful?